In [2]:
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset, SVD, accuracy
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os
import missingno as msno
import pickle

Unload the dataset from pickle

In [3]:
df = pickle.load(open('pickeledDataset', 'rb'))

In [4]:
df.head()

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId
0,88128149,nyheter|trondheim,44.359582,Dette er «Kystad-saken» i korte trekk - og sak...,http://adressa.no/nyheter/trondheim/2017/03/17...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-17T17:39:35.000Z,1490050820,ff9704d73150a5f14bbad454f4f6e0f252f49521
1,1470946317,nyheter|okonomi,34.000000,Han er verdens tredje yngste dollarmilliardær,http://adressa.no/nyheter/okonomi/2017/03/20/h...,cx:i7m47c8k5538rd2u:sdl67619goo2,2017-03-20T21:22:42.000Z,1490050834,8aa45aba5d8e2880410040c0f3c10c7a1e447475
2,620605796,100sport|idrettspolitikk,44.359582,Her er den nye strategien etter VM-nederlaget,http://adressa.no/100sport/idrettspolitikk/her...,cx:1359724185772431277628:1bo1nngcb9hcr,2017-03-20T19:55:59.000Z,1490050836,dff1b689590e3ba0ce21a88d3295af016c3abefd
3,1543770103,pluss|nyheter,9.000000,Ottervik mener Olsø burde informert om avtalen...,http://adressa.no/pluss/nyheter/2017/03/20/ott...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-20T20:28:02.000Z,1490050840,8dc005abfab091585891b0af8c9d9a627e4a2729
4,806901198,pluss|nyheter,44.359582,- I en så tragisk sak er alle tapere,http://adressa.no/pluss/nyheter/2017/03/20/i-e...,cx:y3ojtj1dah373vh26yiostnfa:3bnuogklc05m0,2017-03-20T18:20:54.000Z,1490050845,9924065d1af706392a6d7bdaab34c3ba04259158


Choose the relevant columns to build the recommender system

In [8]:
data = df[['userId', 'url', 'activeTime']]
data.head()

,userId,url,activeTime
0,cx:13675964253951224092221:1wgstmuzo9vjl,http://adressa.no/nyheter/trondheim/2017/03/17...,44.359582
1,cx:i7m47c8k5538rd2u:sdl67619goo2,http://adressa.no/nyheter/okonomi/2017/03/20/h...,34.000000
2,cx:1359724185772431277628:1bo1nngcb9hcr,http://adressa.no/100sport/idrettspolitikk/her...,44.359582
3,cx:13675964253951224092221:1wgstmuzo9vjl,http://adressa.no/pluss/nyheter/2017/03/20/ott...,9.000000
4,cx:y3ojtj1dah373vh26yiostnfa:3bnuogklc05m0,http://adressa.no/pluss/nyheter/2017/03/20/i-e...,44.359582


Normalize the activetime by using StandardScaler

In [10]:
scaler = StandardScaler()
data['activeTime'] = scaler.fit_transform(np.array(data['activeTime']).reshape(-1, 1))
data.head()

<ipython-input-10-c75b81c8ed0b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['activeTime'] = scaler.fit_transform(np.array(data['activeTime']).reshape(-1, 1))


,userId,url,activeTime
0,cx:13675964253951224092221:1wgstmuzo9vjl,http://adressa.no/nyheter/trondheim/2017/03/17...,-0.289087
1,cx:i7m47c8k5538rd2u:sdl67619goo2,http://adressa.no/nyheter/okonomi/2017/03/20/h...,-0.451865
2,cx:1359724185772431277628:1bo1nngcb9hcr,http://adressa.no/100sport/idrettspolitikk/her...,-0.289087
3,cx:13675964253951224092221:1wgstmuzo9vjl,http://adressa.no/pluss/nyheter/2017/03/20/ott...,-0.844684
4,cx:y3ojtj1dah373vh26yiostnfa:3bnuogklc05m0,http://adressa.no/pluss/nyheter/2017/03/20/i-e...,-0.289087


NameError: name 'data' is not defined

In [11]:
reader = Reader()
data = Dataset.load_from_df(data, reader)

Split the dataset into train and test

In [12]:
trainset, testset = train_test_split(data, test_size=0.25)

Create a model on the train dataset

In [13]:
algo = SVD(n_factors=100)
algo.fit(trainset)

Calculate the prediction on testset by using the trained model from earlier step

In [14]:
predictions = algo.test(testset)

MSE measure outcome

In [16]:
accuracy.mse(predictions)

MSE: 1.9765


1.976480364897603

In [ ]:
150: 1.9272218867836675

Visualize the predicted outcomes

In [24]:
pred = pd.DataFrame(predictions)

In [25]:
pred['est'] = scaler.inverse_transform(pred['est'])

In [26]:
predAr = np.array(pred)

Get the top n recommended articles

In [31]:
from collections import defaultdict
def get_top_n(predictions, n=3):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for pred in predictions:
        top_n[pred[0]].append((pred[1], pred[3]))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [32]:
top_n = get_top_n(predAr, n=3)

Print the recommended items for each user

In [34]:

for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid,_) in user_ratings])

cx:1308583952725345187693:udud1r6r2t9x ['http://adressa.no/nyheter/trondheim/2017/03/31/her-er-%c3%a5rets-russeknuter-14535406.ece', 'http://adressa.no/meninger/2017/02/20/de-fleste-private-barnehager-er-ikke-i-n%c3%a6rheten-av-%c3%a5-kunne-ta-ut-overskudd.-det-er-en-evig-kamp-for-%c3%a5-f%c3%a5-budsjettet-i-balanse-14286568.ece', 'http://adressa.no/nyheter/nordtrondelag/2017/01/12/gutt-fikk-trestolpe-i-hodet-p%c3%a5-skolen-14051263.ece']
cx:i29htb5aoe26692l:1rt4ul0023dh4 ['http://adressa.no/pluss/magasin/2017/03/25/mister-frati-14496647.ece', 'http://adressa.no/pluss/magasin/2017/01/14/kan-du-gjette-hvor-dette-er-14056592.ece', 'http://adressa.no/pluss/magasin/2017/03/25/fra-stillhetens-hav-til-kjernekraftverk-14490053.ece']
cx:ibwazmhun5896ylh:2nchnkenka1bd ['http://adressa.no/nyheter/utenriks/2017/01/16/d%c3%b8dstallene-etter-flystyrten-i-kirgisistan-stiger-14066153.ece', 'http://adressa.no/nyheter/moreromsdal/2017/01/08/person-funnet-d%c3%b8d-p%c3%a5-sula-14032916.ece', 'http://adr

cx:in1upprlodq34cnv:2bks4sf6hsrfx ['http://adressa.no/pluss/magasin/2017/02/02/trondheim-i-1977-14141841.ece', 'http://adressa.no/100sport/fotball/derfor-mener-han-rbk-bor-kjope-flere-rene-forsterkninger-228696b.html', 'http://adressa.no/nyheter/nordtrondelag/2017/03/21/unormalt-r%c3%b8ntgenbilde-ble-ikke-fulgt-opp-14-%c3%a5ring-d%c3%b8de-14478788.ece']
cx:3sgcg1210q0sf28li234a7r2se:1u02mckwi64sp ['http://adressa.no/pluss/magasin/2017/01/16/hvor-i-all-verden...-14066667.ece', 'http://adressa.no/bil/slik-bytter-du-bilbatteriet-selv-9799b.html', 'http://adressa.no/nyheter/trondheim/2017/03/14/kan-ikke-forst%c3%a5-at-det-er-et-marked-for-%c3%a5-kj%c3%b8pe-seg-burger-klokka-05-om-morran-14437799.ece']
cx:i5twuir0akye0wqf:15fn9728tiosp ['http://adressa.no/pluss/magasin/2017/03/11/jeg-tok-med-en-bok-om-is-p%c3%a5-flyet-til-usa.-det-skulle-jeg-aldri-ha-gjort-14430255.ece', 'http://adressa.no/nyheter/nordtrondelag/2017/01/17/sn%c3%a5samannen-tror-trump-blir-skutt-14076876.ece', 'http://adressa